In [1]:
import scipy as sp
from scipy import stats
import numpy as np 
import pandas as pd
from FileExporter import create_download_link
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from msresist.mining_bioreplicates import MergeDfbyMean, AvsBacrossCond, AvsBvsCacrossCond
from msresist.plsr import FilteringOutPeptides, MeasuredVsPredicted_LOOCVplot
from msresist.comp_estimator import MyOwnKMEANS
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from msresist.sequence_analysis import Distance, GenerateFastaFile
from Bio import SeqIO, AlignIO
import re

In [2]:
# Varaibles: X phoshopeptides
X = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=0)
header = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=None).iloc[0,:]
treatments = pd.read_csv('./msresist/data/20180817_JG_AM_TMT10plex_R1_psms.csv', header=None).iloc[0,2:]

PN = []
x = list(map(lambda  v : PN.append(v), X.iloc[:,1]))

X_protnames = []
x = list(map(lambda  v : X_protnames.append(v.split("OS")[0]), X.iloc[:,1]))

pre_seqs = list(X.iloc[:,0])

X_seqs = []
for seq in pre_seqs:
    if seq[0] == "y" and "y" not in seq[1:]:
        X_seqs.append(seq.split("-")[0])
    else:
        X_seqs.append(seq[1:].split("-")[0])
    
X_data = X.iloc[:,2:]

# Filtering out insignificant phosphopeptides 
Xf = FilteringOutPeptides(X, header)
Xf_data = np.transpose(Xf.iloc[:, 2:])
Xf_seqs = Xf.iloc[:,0]
Xf_protnames = Xf.iloc[:,1]

# Observations: Y
Y = pd.read_csv('./msresist/data/ydata.csv', header=0)
Y_cv = Y.iloc[:10, 2]

# X[X["Master Protein Descriptions"].str.contains("Calmodulin")]


In [3]:
# DictProtNameToPhospho = dict(zip(X_protnames, X_seqs))

In [4]:
DictProtNameToPhospho = {}
seq1 , seq2, seq3, seq4, seq5, seq6, = [], [], [], [], [], []
for i, seq in enumerate(X_seqs):
    if "y" in seq and "t" not in seq and "s" not in seq:
        seq1.append(seq)
    if "t" in seq and "y" not in seq and "s" not in seq:
        seq2.append(seq)
        DictProtNameToPhospho["t: "] = seq2
    if "s" in seq and "y" not in seq and "t" not in seq:
        seq3.append(seq)
        DictProtNameToPhospho["s: "] = seq3
    if "y" in seq and "t" in seq and "s" not in seq:
        seq4.append(seq)
        DictProtNameToPhospho["y/t: "] = seq4
    if "y" in seq and "s" in seq and "t" not in seq:
        seq5.append(seq)
        DictProtNameToPhospho["y/s: "] = seq5
    if "t" in seq and "s" in seq and "y" not in seq:
        seq6.append(seq)

DictProtNameToPhospho["y: "] = seq1
DictProtNameToPhospho["t: "] = seq2
DictProtNameToPhospho["s: "] = seq3
DictProtNameToPhospho["y/t: "] = seq4
DictProtNameToPhospho["y/s: "] = seq5
DictProtNameToPhospho["t/s: "] = seq6

SeqsBySites = pd.DataFrame(dict([ (k, pd.Series(v)) for k,v in DictProtNameToPhospho.items() ]))

,t:,y/t:,y/s:,s:,y:,t/s:
0,AVPSGAStGIYEALELRDNDkTR,YSQEEITQGFEETGDTLyAPYStHFQLQNQPPQk,ENTPLIGREsPPPSyTSSMR,GEENIEVYHsYFWPLEWTIPSR,AVPSGASTGIyEALELR,NaN
1,YSQEEITQGFEETGDtLYAPYSTHFQLQNQPPQk,ADPEHDHTGFLtEyVATR,IHQQPQSSsPVyGSSAk,EPNVsYIcSR,AVPSGASTGIyEALELRDNDk,NaN
2,YNIANtV,GEGtyGVVYk,YEDGYPGGSDNyGsLSR,YEDGYPGGsDNYGSLSR,DHQPLTEASyVNLPTIALcNTDSPLR,NaN
3,LEDDPSDPtYTSSLGGk,kPPYtDyVSTR,SSARPGGLGSSsLyGLGASRPR,EEENIYsVPHDSTQGk,DyDTLSLR,NaN
4,LISEEELGASVASLSSENPYAtIR,AcTNFMMtPyVVTR,DsYENLSADGEVLHTQGPVDGSLyAk,YsPYDMLESIR,EDGSVIDyELIDQDAR,NaN
5,VHLENATEYAtLR,AGTSFMMtPyVVTR,NaN,AQQPLSLVGcEVVPDPSPDHLYsFR,EERPTFDYLQSVLDDFYTATEGQyQQQP,NaN
6,NHLQADNFYQtV,ADPDHDHTGFLtEyVATR,NaN,NaN,EFAEyASVDR,NaN
7,AFSNLFGGEPLSYtR,NaN,NaN,NaN,EGSDVANAVLDGADcIMLSGETAkGDyPLEAVR,NaN
8,VLEDFFTAtEGQYQPQP,NaN,NaN,NaN,ESGPDLRyEVTSGGGGTSR,NaN
9,TNPFYNtM,NaN,NaN,NaN,ITNNQyEIV,NaN


In [5]:
GenerateFastaFile("./msresist/data/MS_seqs.fa", PN, X_seqs)

In [6]:
MS_seqs = open( "./msresist/data/MS_seqs.fa", "r")
proteome = open('./msresist/data/proteome_uniprot.fa', "r")

In [7]:
DictProtToSeq_UP = {}
for rec2 in SeqIO.parse(proteome, "fasta"):
    UP_seq = str(rec2.seq)
    UP_name = rec2.description.split("HUMAN ")[1].split(" OS")[0]
    DictProtToSeq_UP[UP_name] = str(UP_seq)

In [8]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if valueToFind in item[1]:
            listOfKeys.append(item[0])
    return  listOfKeys

l = getKeysByValue(DictProtToSeq_UP, "YQWINELSSPETR")

print(l[0])
#list(DictProtToSeq_UP.keys())[list(DictProtToSeq_UP.values()).index("YQWINELSSPETR")]

CCR4-NOT transcription complex subunit 9


In [9]:
FileHandle = open("./msresist/data/MS_seqs_matched.fa", "w+")
#counter = 0
for rec1 in SeqIO.parse(MS_seqs, "fasta"):
    MS_seq = str(rec1.seq)
    MS_seqU = str(rec1.seq.upper())
    MS_name = str(rec1.description.split(" OS")[0])
    try:
        UP_seq = DictProtToSeq_UP[MS_name]
        FileHandle.write(">" + MS_name)
        FileHandle.write("\n")
        FileHandle.write(MS_seq)
        FileHandle.write("\n")
    except:
        #counter += 1
        Fixed_name = getKeysByValue(DictProtToSeq_UP, MS_seqU)
        FileHandle.write(">" + Fixed_name[0])
        FileHandle.write("\n")
        FileHandle.write(MS_seq)
        FileHandle.write("\n")       
FileHandle.close()

MS_seqs_matched = open( "./msresist/data/MS_seqs_matched.fa", "r")
#print(counter)

In [10]:
ExtSeqs = []
MS_names = []
counter = 0
for rec1 in SeqIO.parse(MS_seqs_matched, "fasta"):
    MS_seq = str(rec1.seq)
    MS_seqU = str(rec1.seq.upper())
    MS_name = str(rec1.description.split(" OS")[0])
    MS_names.append(MS_name)
    try:
        UP_seq = DictProtToSeq_UP[MS_name] 
        if MS_seqU in UP_seq and MS_name == list(DictProtToSeq_UP.keys())[list(DictProtToSeq_UP.values()).index(UP_seq)]:
            counter += 1
            regexPattern = re.compile(MS_seqU)
            MatchObs = regexPattern.finditer(UP_seq)
            indices = []
            for i in MatchObs:
                indices.append(i.start())   #VHLENATEYAtLR   #YNIANtV
                indices.append(i.end())                 
            if "y" in MS_seq and "t" not in MS_seq and "s" not in MS_seq:
                y_idx = MS_seq.index("y") + indices[0]
                ExtSeqs.append(UP_seq[y_idx-5:y_idx] + "y" + UP_seq[y_idx+1:y_idx+6])
                
            if "t" in MS_seq and "y" not in MS_seq and "s" not in MS_seq:
                t_idx = MS_seq.index("t") + indices[0]
                ExtSeqs.append(UP_seq[t_idx-5:t_idx] + "t" + UP_seq[t_idx+1:t_idx+6])
                
            if "s" in MS_seq and "y" not in MS_seq and "t" not in MS_seq:
                s_idx = MS_seq.index("s") + indices[0]
                ExtSeqs.append(UP_seq[s_idx-5:s_idx] + "s" + UP_seq[s_idx+1:s_idx+6])
                
            if "y" in MS_seq and "t" in MS_seq and "s" not in MS_seq:     
                y_idx = MS_seq.index("y") + indices[0]                  
                ExtSeq = UP_seq[y_idx-5:y_idx] + "y" + UP_seq[y_idx+1:y_idx+6]
                y_idx = MS_seq.index("y")
                if "t" in MS_seq[y_idx-5:y_idx+6]:
                    t_idx = MS_seq[y_idx-5:y_idx+6].index("t")
                    ExtSeqs.append(ExtSeq[:t_idx] + "t" + ExtSeq[t_idx+1:])
                else:
                    ExtSeqs.append(ExtSeq)
                
            if "y" in MS_seq and "s" in MS_seq and "t" not in MS_seq:
                y_idx = MS_seq.index("y") + indices[0]
                ExtSeq = UP_seq[y_idx-5:y_idx] + "y" + UP_seq[y_idx+1:y_idx+6]
                y_idx = MS_seq.index("y")
                if "s" in MS_seq[y_idx-5:y_idx+6]:
                    s_idx = MS_seq[y_idx-5:y_idx+6].index("s")
                    ExtSeqs.append(ExtSeq[:s_idx] + "s" + ExtSeq[s_idx+1:])
                else:
                    ExtSeqs.append(ExtSeq)
                
            if "t" in MS_seq and "s" in MS_seq and "y" not in MS_seq:
                t_idx = MS_seq.index("t") + indices[0]
                ExtSeq = UP_seq[t_idx-5:t_idx] + "t" + UP_seq[t_idx+1:t_idx+6]
                t_idx = MS_seq.index("t")
                if "s" in MS_seq[t_idx-5:t_idx+6]:
                    s_idx = MS_seq[t_idx-5:t_idx+6].index("s")
                    ExtSeqs.append(ExtSeq[:s_idx] + "s" + ExtSeq[s_idx+1:])
                else:
                    ExtSeqs.append(ExtSeq)
    except:
        print("find and replace", MS_name, "in proteome_uniprot.txt. Use: ", MS_seq)
        pass

In [11]:
# # seq1 = 'ENTPLIGREsPPPSyTSSMR'
# seq1 = 'ENTPLIGREsPPPSyTSSS'
# seq2 = seq1.upper()
# y_idx = seq1.index("y")
# ExtSeq = seq2[y_idx-5:y_idx] + "y" + seq2[y_idx+1:y_idx+6]
# print(ExtSeq)
# s_idx = seq1[y_idx-5:y_idx+6].index("s")
# ExtSeq = ExtSeq[:s_idx] + "s" + ExtSeq[s_idx+1:]
# print(ExtSeq)

# if "s" in seq1[y_idx-5:y_idx+6]:
#     print("check")

# # # y_idx = seq1.index("y")
# # # s_idx = seq1.index("s") 
# # # print(ExtSeq)
# # # ExtSeq = ExtSeq[:s_idx] + "s" + ExtSeq[s_idx+1:]

SPPPSyTSSS
sPPPSyTSSS
check


In [12]:
GenerateFastaFile("./msresist/data/BR1_ExtSeqs.fa", MS_names, ExtSeqs)